In [211]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [210]:

import pandas as pd
import numpy as np
import openmdao.api as om

In [212]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [213]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        self.add_input('cog_structures', shape = (2,))
        self.add_input('systems_CG')
        
        self.add_output('m')
        prob.model.add_objective('m')
        
        self.add_output('total_CG', shape = (1,2))
        
    def setup_partials(self):
        self.declare_partials('*', '*', method = 'fd')
        
    def compute(self,inputs,outputs):
        #calculation of mass from systems and structures
        mass = inputs['systems_mass'] + inputs['weight_structures']
        
        outputs['m'] = mass
        
        #calculation of CG from systems and structures
        struct_CG = inputs['cog_structures']
        sys_CG = inputs['systems_CG']
        
        struct_comp_x = struct_CG[0]*inputs['weight_structures']
        struct_comp_y = struct_CG[1]*inputs['weight_structures']
        
        sys_comp_x = sys_CG*inputs['systems_mass']
        
        total_CG_x = (struct_comp_x + sys_comp_x)/mass
        total_CG_y = struct_comp_y/mass
        
        total_CG = [total_CG_x, total_CG_y]
        outputs['total_CG'] = total_CG

In [214]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
#%run aero/aerosetup_2.ipynb
class aero(om.Group):
    pass
class stab(om.Group):
    pass


In [215]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [217]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)
prob.model.set_input_defaults('m', val = 240000)


prob.model.add_design_var('tank_ratio', lower = 0.3, upper = 1)
prob.model.set_input_defaults('tank_ratio', val = 1)


In [218]:
#declare problem options (driver, optimiser)

#prob.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.nonlinear_solver.options['iprint'] = 2
prob.model.nonlinear_solver.options['maxiter'] = 200

#prob.nonlinear_solver.options['maxiter'] = 100

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-5

#setup problem to be run
prob.setup()

In [219]:
prob.run_model()

NL: NLBGS 1 ; 135241144 1
NL: NLBGS 2 ; 47280.556 0.000349601864
NL: NLBGS 3 ; 31923.7097 0.000236050279
NL: NLBGS 4 ; 11611.2383 8.58558127e-05
NL: NLBGS 5 ; 3597.55397 2.66010318e-05
NL: NLBGS 6 ; 1065.92789 7.88168351e-06
NL: NLBGS 7 ; 311.796381 2.30548464e-06
NL: NLBGS 8 ; 90.8651432 6.71874997e-07
NL: NLBGS 9 ; 26.4516972 1.95589126e-07
NL: NLBGS 10 ; 7.69791385 5.69199108e-08
NL: NLBGS 11 ; 2.24002451 1.65631881e-08
NL: NLBGS 12 ; 0.651809848 4.81961206e-09
NL: NLBGS 13 ; 0.189664346 1.40241601e-09
NL: NLBGS 14 ; 0.0551886154 4.08075633e-10
NL: NLBGS 15 ; 0.0160587972 1.1874195e-10
NL: NLBGS 16 ; 0.00467279199 3.45515562e-11
NL: NLBGS Converged


In [220]:
#prob.run_driver()

In [221]:
prob.model.get_design_vars()

{'tank_ratio': {'scaler': None,
  'adder': None,
  'name': 'tank_ratio',
  'upper': 1.0,
  'lower': 0.3,
  'ref': None,
  'ref0': None,
  'units': None,
  'cache_linear_solution': False,
  'indices': None,
  'flat_indices': False,
  'parallel_deriv_color': None,
  'source': '_auto_ivc.v2',
  'orig': (None, 'tank_ratio'),
  'distributed': False,
  'total_adder': None,
  'total_scaler': None,
  'size': 1,
  'global_size': 1}}

In [222]:
prob.model.get_val('tank_ratio')

array([1.])

In [223]:
#get properties of specific constants in disciplines
#prob.model.sys.options._dict['pp_mass']

prob.model.list_outputs()

35 Explicit Output(s) in 'model'

varname                         val                 
------------------------------  --------------------
sys
  fuel_mass_calc
    fuel_mass                   [19336.54240762]    
  tanks
    tank1_mass_full             [14954.31925929]    
    tank1_mass_empty            [3352.39381472]     
    tank2_mass_full             [10319.12923167]    
    tank2_mass_empty            [2584.51226862]     
    CGtp                        [26.90188178]       
    tank1_x                     [7.24171317]        
    tank2_x                     [12.87728983]       
    tank_config_n               [5.]                
    tank_i_t                    [0.14881226]        
    length_fuselage             [65.27115332]       
  pipes
    pipe_CG                     [13.95793735]       
    pipe_mass                   [385.25431126]      
    pipe_i_t                    [0.04628407]        
  engine
    engine_mass                 [203.90823463]      
    engine_x       

[('sys.fuel_mass_calc.fuel_mass', {'val': array([19336.54240762])}),
 ('sys.tanks.tank1_mass_full', {'val': array([14954.31925929])}),
 ('sys.tanks.tank1_mass_empty', {'val': array([3352.39381472])}),
 ('sys.tanks.tank2_mass_full', {'val': array([10319.12923167])}),
 ('sys.tanks.tank2_mass_empty', {'val': array([2584.51226862])}),
 ('sys.tanks.CGtp', {'val': array([26.90188178])}),
 ('sys.tanks.tank1_x', {'val': array([7.24171317])}),
 ('sys.tanks.tank2_x', {'val': array([12.87728983])}),
 ('sys.tanks.tank_config_n', {'val': array([5.])}),
 ('sys.tanks.tank_i_t', {'val': array([0.14881226])}),
 ('sys.tanks.length_fuselage', {'val': array([65.27115332])}),
 ('sys.pipes.pipe_CG', {'val': array([13.95793735])}),
 ('sys.pipes.pipe_mass', {'val': array([385.25431126])}),
 ('sys.pipes.pipe_i_t', {'val': array([0.04628407])}),
 ('sys.engine.engine_mass', {'val': array([203.90823463])}),
 ('sys.engine.engine_x', {'val': array([14.82308546])}),
 ('sys.engine.engine_y', {'val': array([18.])}),
 

In [ ]:
mass = 6
naca_series = '23014'
max_allowable_wing_span = 90
keyaeronumbers = pd.DataFrame(list([mass,naca_series,max_allowable_wing_span]))
keyaeronumbers

In [ ]:
keyaeronumbers = pd.read_csv("keyAero.dat")


In [ ]:
f = pd.DataFrame(keyaeronumbers)

In [ ]:
str(int(f.iloc[1]))